In [108]:
import pandas as pd
import networkx as nx
import datetime
import matplotlib.pyplot as plt

In [109]:
df = pd.read_csv('fourCollections.csv', dtype={'collection_slug': 'str', 'asset_id': 'int', 'asset_name': 'str', 'owner_username': 'str', 'owner_address': 'str', 'event_type': 'str'})

In [110]:
# Clean Data
df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('collection_slug', axis=1,inplace=True)
df.drop('asset_contract_date', axis=1, inplace=True)
pd.to_datetime(df['event_timestamp']).dt.strftime('%m/%d/%Y %H:%M')
df.drop_duplicates(keep='first')
df = df[df['event_type'] == 'successful']
df.head()

,collection_name,asset_id,asset_name,owner_username,owner_address,event_timestamp,event_type
142798,Azuki,206855934,Azuki #9442,Fandoghi,0x489795dc70252cf420177f6ca9a3feb2195a34dd,2022-06-17T10:04:20,successful
142799,Azuki,204242783,Azuki #1567,NaN,0x4328b6537112fc8cc216251712d6b0d7f191bee9,2022-06-17T08:55:39,successful
142800,Azuki,204242812,Azuki #1610,NONAME789,0x2791b3fe4690799fa519eeb0cb7319c7a90da828,2022-06-17T04:43:22,successful
142801,Azuki,204258025,Azuki #5902,NaN,0x709c43d68e4045823edf3f59bf31c4fa307cef2f,2022-06-17T04:09:09,successful
142802,Azuki,204246264,Azuki #1983,NaN,0x4bbc35c3cb2a5a65c7427012a9b1277d93751c32,2022-06-17T02:58:18,successful


In [116]:
# Sorts addresses with more than 1 asset

az = 'Azuki'
ba = 'Bored Ape Yacht Club'
mf = 'mfers'
cc = 'Crypto Coven'

a = df[df['owner_address'].isin(df['owner_address'].value_counts()[df['owner_address'].value_counts() > 1].index)].owner_address.unique()

df_a = pd.DataFrame()
df_b = pd.DataFrame()

i = 0
for owner in a:
    temp = df[df['owner_address'] == a[i]]

    temp.reset_index(inplace=True)
    address = temp.owner_address[0]

    df_a = df_a.append(temp)

    mf_count = (temp[temp['collection_name'] == 'mfers'].count())
    m = mf_count['collection_name']

    cc_count = (temp[temp['collection_name'] == 'Crypto Coven'].count())
    c = cc_count['collection_name']

    if (m == 2 & c == 1):
        weight = '001'
    elif (c == 3):
        weight = '002'
    else:
        weight = '003'
        

    df_b.append({'address': address, 'weight': weight}, ignore_index=True)
    
    i += 1
    break


# Sort Data
'''
df_b = df_a[(df_a['event_timestamp'] > '2021-12-20T10:00') & (df_a['event_timestamp'] < '2021-12-21T12:00')]'''

"\ndf_b = df_a[(df_a['event_timestamp'] > '2021-12-20T10:00') & (df_a['event_timestamp'] < '2021-12-21T12:00')]"

In [117]:

df_b

""


In [113]:
'''
let's define nodes and links: each address is a node, the more total nfts owned from all collection the larger the node; 
if two address co-own any nfts from the same collection add a link with weight 1:
001 1 witch 2 mfers
002 3 witch
003 5 witch 1 mfers-
link1: 001-002, weight 1
link2: 001-003, weight 2
link3: 002-003, weight 1
'''

"\nlet's define nodes and links: each address is a node, the more total nfts owned from all collection the larger the node; \nif two address co-own any nfts from the same collection add a link with weight 1:\n001 1 witch 2 mfers\n002 3 witch\n003 5 witch 1 mfers-\nlink1: 001-002, weight 1\nlink2: 001-003, weight 2\nlink3: 002-003, weight 1\n"